In [24]:
import cv2

In [25]:
import sys
import os

In [26]:
import numpy as np
import pandas as pd

In [27]:
os.chdir('/home/shuzh/TidyBC/7_AtypicalMatching/lehmann')

In [28]:
UNS = cv2.imread("UNS.png")

In [29]:
labels = cv2.imread('labels.png')
EGFR = cv2.imread('EGFR.png')
MKI67 = cv2.imread('MKI67.png')
MYC = cv2.imread('MYC.png')
SNAI2 = cv2.imread('SNAI2.png')
CDH1 = cv2.imread('CDH1.png')
TWIST1 = cv2.imread('TWIST1.png')

In [98]:
labels.shape

(18, 1090, 3)

In [30]:
cut0 = 0 # UNS
cut1 = 138 # BL1
cut2 = 328 # BL2
cut3 = 483 # IM
cut4 = 687 # M
cut5 = 888 # MSL
cut6 = 965 # LAR

In [31]:
cuts = np.array([138,328,483,687,888,965])

In [32]:
def get_green_red_channels(img):
    img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    mask_green = cv2.inRange(img_hsv, (36, 25, 25), (70, 255,255))
    mask_red1 = cv2.inRange(img_hsv, (0, 70, 50), (10, 255, 255))
    mask_red2 = cv2.inRange(img_hsv, (170, 70, 50), (180, 255, 255))
    mask_orange = cv2.inRange(img_hsv, (10, 100, 20), (25, 255, 255))
    mask_yellow = cv2.inRange(img_hsv, (21, 39, 64), (40, 255, 255))
    
    imask_red1 = mask_red1>0
    imask_red2 = mask_red2>0
    imask_orange = mask_orange>0
    imask_yellow = mask_yellow>0
    red = np.zeros_like(img, np.uint8)
    red[imask_red1] = img[imask_red1]
    red[imask_red2] = img[imask_red2]
    red[imask_orange] = img[imask_orange]
    red[imask_yellow] = img[imask_yellow]
    
    imask_green = mask_green>0
    green = np.zeros_like(img, np.uint8)
    green[imask_green] = img[imask_green]
    
    return red, green
    


In [33]:
red,green = get_green_red_channels(UNS)

In [34]:
green_gray = cv2.cvtColor(green, cv2.COLOR_BGR2GRAY)
red_gray = cv2.cvtColor(red, cv2.COLOR_BGR2GRAY)

In [17]:
cv2.imwrite("red_gray.png", red_gray)

True

In [18]:
cv2.imwrite("green_gray.png", green_gray)

True

In [21]:
red_gray.max()

217

In [22]:
green_gray.max()

243

In [35]:
def get_expression(red, green):
    green_gray = cv2.cvtColor(green, cv2.COLOR_BGR2GRAY)
    red_gray = cv2.cvtColor(red, cv2.COLOR_BGR2GRAY)
    green_std = np.interp(green_gray, (0,green_gray.max()), (0, -2))
    red_std = np.interp(red_gray, (0,red_gray.max()), (0, 2))
    return np.mean(green_std) + np.mean(red_std)

In [36]:
def process_images(img, cuts):
    UNS = img[:, 0:cuts[0], :]
    BL1 = img[:, cuts[0]:cuts[1], :]
    BL2 = img[:, cuts[1]:cuts[2], :]
    IM = img[:, cuts[2]:cuts[3], :]
    M = img[:, cuts[3]:cuts[4], :]
    MSL = img[:, cuts[4]:cuts[5], :]
    LAR = img[:, cuts[5]:, :]
    
    dict_subtypes = {'UNS': UNS, 'BL1': BL1, 'BL2': BL2, 'IM': IM, 'M': M, 'MSL': MSL,'LAR': LAR}
    list_subtypes = list(dict_subtypes.items())
    lst = []
    for i in range(7):
        key, subtype = list_subtypes[i]
        red, green = get_green_red_channels(subtype)
        lst.append(get_expression(red,green))
    return lst

In [ ]:
process_images

In [37]:
EGFR_lst = process_images(EGFR, cuts)
MKI67_lst = process_images(MKI67, cuts)
CDH1_lst = process_images(CDH1, cuts)
MYC_lst = process_images(MYC, cuts)
SNAI2_lst = process_images(SNAI2, cuts)
TWIST1_lst = process_images(TWIST1, cuts)

In [38]:
d = {"EGFR": EGFR_lst, "MKI67": MKI67_lst , "CDH1": CDH1_lst, "MYC": MYC_lst, "SNAI2": SNAI2_lst, "TWIST1": TWIST1_lst}
df = pd.DataFrame(d)

In [39]:
df.index = ['UNS', "BL1", "BL2", "IM","M", "MSL", "LAR"]

In [40]:
df

,EGFR,MKI67,CDH1,MYC,SNAI2,TWIST1
UNS,-0.211543,0.294426,-0.378770,0.297679,-0.223286,-0.423009
BL1,-0.427658,0.299385,0.012041,0.419907,-0.229507,-0.332856
BL2,0.192129,-0.247571,0.195448,-0.093197,0.066660,0.148477
IM,-0.411844,0.150117,0.000014,0.187499,-0.665337,-0.251244
M,-0.002010,0.068957,-0.090739,-0.010692,0.397465,0.158457
MSL,-0.086514,-0.764067,-0.148906,-0.389109,0.370718,0.506418
LAR,0.126756,-0.708365,0.282522,-0.830983,0.040879,-0.220295


In [41]:
df.to_csv("avg_expression_lehmann.csv")